In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [ ]:
# Device is https://datasheets.maximintegrated.com/en/ds/MAX7219-MAX7221.pdf
# 4 banks of 5*4 8x8 on the SPI bus with 4 chip selects
# the modules mqtt_setup.py and ledgridpanel.py are saved lower down

In [4]:
%fetchfile --print config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
boardname    esp32ledstrip

In [7]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   mqtt.local
pinled       2
boardname    esp32LED

Sent 5 lines (111 bytes) to config.txt.


In [87]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [6]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-240-g519746cae on 2019-08-26; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f3b70ee3c88, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [88]:
%sendtofile main.py

import time
from ledgridpanel import fbuff, fbuffwidth, fbuffheight, setbrightness, show
from mqtt_setup import client, ipnumber, boardname, mqttbroker

def textrows(ms):
    fbuff.fill(0)
    for i, m in enumerate(ms):
        fbuff.text(m[:fbuffwidth//8], 0, i*8, 1)
    show()
    
textrows(["board: %s"%boardname, "ip:%s"%ipnumber, "mqttip:%s"%mqttbroker])

msg = None
def callback(topic, lmsg):
    global msg
    msg = lmsg
    print((topic, lmsg))
    
client.set_callback(callback)
client.subscribe(boardname+"/message")

t0 = 0
fbuffcharwidth = fbuffwidth//8 - 1 # (-1 for the extra big from scroll buffer)
while True:
    client.check_msg()
    lmsg = msg
    msg = None
    if lmsg:
        fbuff.fill(0)
        for i in range(0, len(lmsg), fbuffcharwidth):
            fbuff.text(lmsg[i:i+fbuffcharwidth], 0, (i//fbuffcharwidth)*8, 1)
        show()
        t0 = time.ticks_ms()
    
    if time.ticks_ms() > t0 + 5000:
        setbrightness(1)
        time.sleep_ms(50)
        setbrightness(0)
        t0 = time.ticks_ms()


Sent 39 lines (1038 bytes) to main.py.


In [85]:
for i in range(0, len(msg), fbuffcharwidth):
    print(msg[i:i+fbuffcharwidth])



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f82d6a222b0, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [5]:
#%sendtofile ledgridpanel.py

# Very minimal scrolling panel
numlegsegmentsX = const(5)
numledcells = const(numlegsegmentsX*4)
mstimedelay = const(10)
cspinsY = [26]
numlegsegmentsY = len(cspinsY)

# setup and initialize the LED strip
from machine import Pin, SPI
#spi = SPI(1, baudrate=1000000, polarity=0, phase=0)  # ESP8266 one
spi = SPI(1, 1000000, sck=Pin(14), mosi=Pin(13))  # HSPI for ESP32

css = [ Pin(p, Pin.OUT)  for p in cspinsY ]
for cs in css:
    for i in range(0, 10, 2):
        cs.value(0)
        x = b"\x0c\x00\x0f\x00\x0b\x07\x09\x00\x0c\x01"[i:i+2]
        spi.write(x*numledcells)
        cs.value(1)

def setbrightness(b):
    for cs in css:
        cs.value(0)  # 0<=brightness<=15
        spi.write(bytes([0x0a, b])*numledcells)
        cs.value(1)
setbrightness(0)

import framebuf
buffer = bytearray((8*numlegsegmentsY)*(numledcells+1))  # make one extra character to scroll from
fbuffwidth = 8*(numledcells+1)
fbuffheight = 8*numlegsegmentsY
fbuff = framebuf.FrameBuffer(buffer, fbuffwidth, fbuffheight, framebuf.MONO_HLSB)

s = bytearray(8)
def showY(iy):
    cs = css[iy]
    for y in range(8):
        s[6] = s[4] = s[2] = s[0] = y+1
        cs.value(0)  # cannot put in loop as resets the shifting
        for m in range(0, numledcells, 4):
            s[1] = buffer[(y+iy*8)*(numledcells+1) + (numledcells - m - 1)]
            s[3] = buffer[(y+iy*8)*(numledcells+1) + (numledcells - m - 2)]
            s[5] = buffer[(y+iy*8)*(numledcells+1) + (numledcells - m - 3)]
            s[7] = buffer[(y+iy*8)*(numledcells+1) + (numledcells - m - 4)]
            spi.write(s)
        cs.value(1)
        
def show():
    for i in range(numlegsegmentsY):
        showY(i)
    
for i in range(0, numledcells*8, 4):
    for j in range(numlegsegmentsY):
        fbuff.fill_rect(i, (i%8)+j*8, 4, 4, 1)
show()


In [ ]:
%sendtofile --source ../basicsockets/minimalmdns.py

In [29]:
%sendtofile mqtt_setup.py

# mqtt technology
import network, machine, time, ubinascii
from minimalmdns import mdnshostnametoipnumber

fconfig = dict(x.split()  for x in open("config.txt"))
pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print("localip", ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
mqttbroker = fconfig["mqttbroker"]
if mqttbroker[-6:] == ".local":
    from minimalmdns import mdnshostnametoipnumber
    mqttbroker = mdnshostnametoipnumber(si, mqttbroker)
    print(fconfig["mqttbroker"], mqttbroker)
    
boardname = fconfig["boardname"]
client = MQTTClient(boardname, mqttbroker)
client.connect()
client.publish(boardname+"/ip", ipnumber, retain=True, qos=1)
client.publish(boardname+"/mac", ubinascii.hexlify(si.config('mac'),':').decode().upper(), retain=True, qos=1)



Sent 31 lines (1013 bytes) to mqtt_setup.py.


In [ ]:
def callback(topic, msg):
    global message, messagechar, messagepix
    print((topic, msg))
    message = msg
    messagechar, messagepix = -numledcells, 7
client.set_callback(callback)
client.subscribe(fconfig["boardname"]+"/message")

tnext = 0
while True:
    while time.ticks_ms() < tnext:
        pass
    tnext = time.ticks_ms() + mstimedelay

    msgnext()
    if messagechar >= len(message):
        messagechar, messagepix = -numledcells, 7
    client.check_msg()


In [ ]:
def callback(topic, msg):
    global message, messagechar, messagepix
    print((topic, msg))
    message = msg
    messagechar, messagepix = -numledcells, 7
client.set_callback(callback)
client.subscribe(fconfig["boardname"]+"/message")

tnext = 0
while True:
    while time.ticks_ms() < tnext:
        pass
    tnext = time.ticks_ms() + mstimedelay

    msgnext()
    if messagechar >= len(message):
        messagechar, messagepix = -numledcells, 7
    client.check_msg()

    

In [37]:
# junk scrolling code
import time
messages = ["DoESLiverpool is...?", "None in Liverpool.!.!"]
messagechar = -1
messagepix = 7
def msgnext():
    global messagechar, messagepix
    messagepix += 1
    if messagepix < 8:
        fbuff.scroll(-1,0)
    else:
        messagechar += 1
        messagepix = 0
        fbuff.fill(0)
        mc = max(messagechar, 0)
        for y, message in enumerate(messages):
            fbuff.text(message[mc:mc+numledcells+1], -min(0, messagechar)*8, y*8+(messagechar), 1)
    show()

while 1:
    msgnext()
    if messagechar >= len(messages[0]):
        messagechar, messagepix = -numledcells, 7
    #time.sleep_ms(20)



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
AttributeError: 'FrameBuffer' object has no attribute 'width'
